In [1]:
!pip install keras==2.0.1
!pip install git+https://github.com/bstriner/keras-adversarial.git

    100% |████████████████████████████████| 194kB 6.2MB/s 
  Running setup.py bdist_wheel for keras ... - \ done
  Stored in directory: /root/.cache/pip/wheels/c2/be/39/81d0b27e26aef72f012ae54263dda355f75bb1f0c2eff64264
Successfully built keras
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6
  Cloning https://github.com/bstriner/keras-adversarial.git to /tmp/pip-req-build-dgbgv1h2
  Running setup.py bdist_wheel for keras-adversarial ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-hfy9q0ft/wheels/f4/34/ba/6041a92244597803d1f8954d649e7899d011d46f33c02dc476
Successfully built keras-adversarial


In [2]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Reshape


def dim_ordering_fix(x):
    if K.image_dim_ordering() == 'th':
        return x
    else:
        return np.transpose(x, (0, 2, 3, 1))


def dim_ordering_unfix(x):
    if K.image_dim_ordering() == 'th':
        return x
    else:
        return np.transpose(x, (0, 3, 1, 2))


def dim_ordering_shape(input_shape):
    if K.image_dim_ordering() == 'th':
        return input_shape
    else:
        return (input_shape[1], input_shape[2], input_shape[0])


def dim_ordering_input(input_shape, name):
    if K.image_dim_ordering() == 'th':
        return Input(input_shape, name=name)
    else:
        return Input((input_shape[1], input_shape[2], input_shape[0]), name=name)


def dim_ordering_reshape(k, w, **kwargs):
    if K.image_dim_ordering() == 'th':
        return Reshape((k, w, w), **kwargs)
    else:
        return Reshape((w, w, k), **kwargs)


def channel_axis():
    if K.image_dim_ordering() == 'th':
        return 1
    else:
        return 3

Using TensorFlow backend.


In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import keras.backend as K
from keras.layers import Reshape, Flatten, LeakyReLU, Activation, Dense, BatchNormalization, SpatialDropout2D
from keras.layers.convolutional import Conv2D, UpSampling2D, MaxPooling2D, AveragePooling2D
from keras.regularizers import L1L2
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.datasets import cifar10
from keras_adversarial.image_grid_callback import ImageGridCallback
from keras_adversarial import AdversarialModel, simple_gan, gan_targets, fix_names
from keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling

In [0]:
def model_generator():
    model = Sequential()
    nch = 256
    reg = lambda: L1L2(l1=1e-7, l2=1e-7)
    h = 5
    model.add(Dense(nch * 4 * 4, input_dim=100, kernel_regularizer=reg()))
    model.add(BatchNormalization())
    model.add(Reshape(dim_ordering_shape((nch, 4, 4))))
    model.add(Conv2D(int(nch / 2), (h, h), padding="same", kernel_regularizer=reg()))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(int(nch / 2), (h, h), padding="same", kernel_regularizer=reg()))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(int(nch / 4), (h, h), padding="same", kernel_regularizer=reg()))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(3, (h, h), padding="same", kernel_regularizer=reg()))
    model.add(Activation("sigmoid"))
    return model

In [0]:
def model_discriminator():
    nch = 256
    h = 5
    reg = lambda: L1L2(l1=1e-7, l2=1e-7)

    c1 = Conv2D(int(nch / 4),
                (h, h),
                padding="same",
                kernel_regularizer=reg(),
                input_shape=dim_ordering_shape((3, 32, 32)))
    c2 = Conv2D(int(nch / 2),
                (h, h),
                padding="same",
                kernel_regularizer=reg())
    c3 = Conv2D(nch,
                (h, h),
                padding="same",
                kernel_regularizer=reg())
    c4 = Conv2D(1,
                (h, h),
                padding="same",
                kernel_regularizer=reg())

    def m(dropout):
        model = Sequential()
        model.add(c1)
        model.add(SpatialDropout2D(dropout))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(LeakyReLU(0.2))
        model.add(c2)
        model.add(SpatialDropout2D(dropout))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(LeakyReLU(0.2))
        model.add(c3)
        model.add(SpatialDropout2D(dropout))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(LeakyReLU(0.2))
        model.add(c4)
        model.add(AveragePooling2D(pool_size=(4, 4), padding="valid"))
        model.add(Flatten())
        model.add(Activation("sigmoid"))
        return model
    return m

In [0]:
def cifar10_process(x):
    x = x.astype(np.float32) / 255.0
    return x


def cifar10_data():
    (xtrain, ytrain), (xtest, ytest) = cifar10.load_data()
    return cifar10_process(xtrain), cifar10_process(xtest)

In [0]:
def example_gan(adversarial_optimizer, path, opt_g, opt_d, nb_epoch,
                generator, discriminator, latent_dim,
                targets=gan_targets, loss="binary_crossentropy"):
    csvpath = os.path.join(path, "history.csv")
    if os.path.exists(csvpath):
        print("Already exists: {}".format(csvpath))
        return

    print("Training: {}".format(csvpath))
    # gan (x - > yfake, yreal), z is gaussian generated on GPU
    # can also experiment with uniform_latent_sampling
    d_g = discriminator(0)
    d_d = discriminator(0.5)
    generator.summary()
    d_d.summary()
    gan_g = simple_gan(generator, d_g, None)
    gan_d = simple_gan(generator, d_d, None)
    x = gan_g.inputs[1]
    z = normal_latent_sampling((latent_dim,))(x)
    # estiminate z from inputs
    gan_g = Model([x], fix_names(gan_g([z, x]), gan_g.output_names))
    gan_d = Model([x], fix_names(gan_d([z, x]), gan_d.output_names))

    # build adversarial model
    model = AdversarialModel(player_models=[gan_g, gan_d],
                             player_params=[generator.trainable_weights,
                                            d_d.trainable_weights],
                             player_names=["generator", "discriminator"])
    model.adversarial_compile(adversarial_optimizer=adversarial_optimizer,
                              player_optimizers=[opt_g, opt_d],
                              loss=loss)

    # create callback to generate images
    zsamples = np.random.normal(size=(10 * 10, latent_dim))

    def generator_sampler():
        xpred = generator.predict(zsamples)
        xpred = dim_ordering_unfix(xpred.transpose((0, 2, 3, 1)))
        return xpred.reshape((10, 10) + xpred.shape[1:])

    generator_cb = ImageGridCallback(
                    os.path.join(path, "epoch-{:03d}.png"),
                    generator_sampler, cmap=None)

    callbacks = [generator_cb]
    if K.backend() == "tensorflow":
        callbacks.append(
            TensorBoard(log_dir=os.path.join(path, "logs"),
                        histogram_freq=0, write_graph=True, write_images=True))

    # train model
    xtrain, xtest = cifar10_data()
    y = targets(xtrain.shape[0])
    ytest = targets(xtest.shape[0])
    history = model.fit(x=xtrain, y=y, validation_data=(xtest, ytest),
                        callbacks=callbacks, epochs=nb_epoch,
                        batch_size=32)

    # save history to CSV
    df = pd.DataFrame(history.history)
    df.to_csv(csvpath)

    # save models
    generator.save(os.path.join(path, "generator.h5"))
    d_d.save(os.path.join(path, "discriminator.h5"))

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-10-31 16:06:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.204.22.7, 34.196.224.14, 52.20.145.121, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.204.22.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  9.64MB/s    in 0.5s    

2018-10-31 16:06:32 (9.64 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [11]:
get_ipython().system_raw('tensorboard --logdir ./output/gan-cifar10/logs --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &') # run nrok to tunnel tensorboard to outside world

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://c6bf09bc.ngrok.io


In [10]:
# z \in R^100
latent_dim = 100
# x \in R^{28x28}
# generator (z -> x)
generator = model_generator()
# discriminator (x -> y)
discriminator = model_discriminator()
if not os.path.exists("output"):
    os.mkdir("output")
if not os.path.exists("output/gan-cifar10"):
    os.mkdir("output/gan-cifar10")
example_gan(AdversarialOptimizerSimultaneous(), "output/gan-cifar10",
            opt_g=Adam(1e-4, decay=1e-5),
            opt_d=Adam(1e-3, decay=1e-5),
            nb_epoch=100, generator=generator, discriminator=discriminator,
            latent_dim=latent_dim)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Training: output/gan-cifar10/history.csv
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4096)              413696    
_________________________________________________________________
batch_normalization_1 (Batch (None, 4096)              16384     
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 128)         819328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 128)         512       
_________________________________________________________________
leaky_re_lu_1 (LeakyR

KeyboardInterrupt: ignored